# Semantic segmentation retraining

---

**Author:** [rodoart](https://github.com/rodoart/)<br>
**Date created:** 2021/07/14<br>
**Last modified:** 2021/07/20<br>
**Description:** 
This is an attempt to retrain the last layers of the pspnet_101_voc12 neural network, chosen because it yielded more mean UI in preliminary tests on the previous notebook. We're following this [image-segmentation-keras](https://github.com/divamgupta/image-segmentation-keras).

## Path config

If you want the files to be copied to another folder within the same machine you are working on, by a source path other than remote.

In [ ]:
PROJECT_SLUG = 'vigilancia_mascotas'
NAME = 'semantic_segmentation_retraining'
NUMBER = '1.0'

NOTEBOOK_NAME = f'{NUMBER}-{PROJECT_SLUG}-{NAME}.ipynb'

USING_COLAB = True

if USING_COLAB:
    DRIVE_MOUNT = '/drive'
    REMOTE_PATH = f'{DRIVE_MOUNT}/MyDrive/IA/seminario_innovacion/{PROJECT_SLUG}'
    LOCAL_PATH = '.'
    NOTEBOOK_PATH = f'{DRIVE_MOUNT}/MyDrive/Colab Notebooks/'

else:
    REMOTE_PATH = '..'
    LOCAL_PATH = ''
    DRIVE_MOUNT = ''
    NOTEBOOK_PATH = f'G:\\Mi unidad\\Colab Notebooks\\'

In [ ]:
if USING_COLAB:
    from google.colab import drive
    drive.mount(DRIVE_MOUNT)

Mounted at /drive


## Path functions

In [ ]:
import sys

# It depends on where the library that comes with this package is stored.
sys.path.append(REMOTE_PATH)

In [ ]:
from vigilancia_mascotas.utils.paths \
  import make_two_dir_function, TwoWorkspacePath

In [ ]:
path = make_two_dir_function(
    local_workspace=LOCAL_PATH, 
    remote_workspace=REMOTE_PATH
)

## Prepare the dataset

The data is already prepared. They can be downloaded like this:

In [ ]:
from vigilancia_mascotas.data.make_dataset import DataDownload

In [ ]:
data_object = DataDownload(local_workspace=path().local,
                           remote_workspace=path().remote)
data_object.start()
data_path = data_object.dataset_processed_path

## Fine-tuning from existing segmentation model

In [ ]:
if USING_COLAB:
    !pip install keras==2.4.3
    !pip install tensorflow==2.4.1
    !pip install keras_segmentation   


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires keras<2.9,>=2.8.0rc0, but you have keras 2.4.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.3 MB 13 kB/s 
     |████████████████████████████████| 2.9 MB 51.0 MB/s 
     |████████████████████████████████| 3.8 MB 54.6 MB/s 
     |████████████████████████████████| 462 kB 73.5 MB/s 
     |████████████████████████████████| 14.8 MB 48.5 MB/s 
     |████████████████████████████████| 132 kB 42.6 MB/s 
  Created w

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 13.0 MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-py3-none-any.whl size=29071 sha256=5901a43d7d3fe7860278bf7baf64d23a597234bdd102c2694a75003725df2bd7
  Stored in directory: /root/.cache/pip/wheels/68/dc/b3/596a3e2461ba16e935ef31661c26e823f841cfb577cec4c47a
Successfully built keras-segmentation
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from keras_segmentation.models.model_utils import transfer_weights
from keras_segmentation.pretrained import pspnet_101_voc12
from keras_segmentation.models.pspnet import pspnet_101

In [ ]:
pretrained_model = pspnet_101_voc12()

new_model = pspnet_101( n_classes=30 )

transfer_weights( pretrained_model , new_model  ) # transfer weights from pre-trained model to your model

Copying weights 


412it [00:00, 782.57it/s]

Copied weights of 222 layers and skipped 1 layers


### Paths

In [ ]:
from os import makedirs

In [ ]:
tuple_path = ('data','processed','semantic_segmentation','unity_residential_interiors')

train_images_path = path(*tuple_path, 'train_images')
train_labels_path  = path(*tuple_path, 'train_labels')
val_images_path = path(*tuple_path, 'val_images')
val_labels_path  = path(*tuple_path, 'val_labels')

model_path = path('models', 'pspnet_101', 'run_1')
makedirs(model_path.remote, exist_ok=True)

### Configure training

In [ ]:
new_model.train(
    train_images = str(train_images_path.local),
    train_annotations = str(train_labels_path.local),
    validate=True,
    val_images=str(val_images_path.local),
    val_annotations=str(val_labels_path.local),
    checkpoints_path = str(model_path.remote.joinpath('run_1')),
    auto_resume_checkpoint=True,
    epochs=10,
    do_augment=True
    )

Verifying training dataset


100%|██████████| 700/700 [01:09<00:00, 10.11it/s]


Dataset verified! 
Verifying validation dataset


100%|██████████| 300/300 [00:30<00:00,  9.74it/s]
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Dataset verified! 
Epoch 1/10
  3/512 [..............................] - ETA: 1:16:02 - loss: 3.4916 - accuracy: 0.0166

/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:191: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  segments = segmentation.slic(image, n_segments=n_segments_samples[i], compactness=10)


512/512 [==============================] - 2979s 6s/step - loss: 3.4666 - accuracy: 0.0447 - val_loss: 3.6562 - val_accuracy: 0.0315
saved  /drive/MyDrive/IA/seminario_innovacion/vigilancia_mascotas/models/pspnet_101/run_1/run_1.0
Epoch 2/10
512/512 [==============================] - 2882s 6s/step - loss: 3.1688 - accuracy: 0.1571 - val_loss: 3.4556 - val_accuracy: 0.0602
saved  /drive/MyDrive/IA/seminario_innovacion/vigilancia_mascotas/models/pspnet_101/run_1/run_1.1
Epoch 3/10
512/512 [==============================] - 3069s 6s/step - loss: 2.9994 - accuracy: 0.2222 - val_loss: 3.2726 - val_accuracy: 0.1052
saved  /drive/MyDrive/IA/seminario_innovacion/vigilancia_mascotas/models/pspnet_101/run_1/run_1.2
Epoch 4/10
512/512 [==============================] - 3062s 6s/step - loss: 2.8375 - accuracy: 0.2736 - val_loss: 3.1483 - val_accuracy: 0.1373
saved  /drive/MyDrive/IA/seminario_innovacion/vigilancia_mascotas/models/pspnet_101/run_1/run_1.3
Epoch 5/10
497/512 [=======================

In [ ]:
?new_model.train

## Update notebook.

In [ ]:
#update_notebook('to_remote')
update_notebook()